In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import sklearn
import os
import sys
import time
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow import keras
# print(tf.__version__)
# print(sys.version_info)
# for module in mpl,np,pd,sklearn,tf,keras:
#     print(module.__name__, module.__version__)

In [2]:
fashion_minist = keras.datasets.fashion_mnist
(x_train_all, y_train_all), (x_test, y_test) = fashion_minist.load_data()
x_vaild, x_train = x_train_all[:5000], x_train_all[5000:]
y_vaild, y_train = y_train_all[:5000], y_train_all[5000:]
print(x_vaild.shape, y_vaild.shape)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(5000, 28, 28) (5000,)
(55000, 28, 28) (55000,)
(10000, 28, 28) (10000,)


In [3]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)
x_vaild = scaler.fit_transform(x_vaild.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)
x_test = scaler.fit_transform(x_test.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)

In [4]:
print(np.max(x_train),np.min(x_train))

2.0231433 -0.8105136


In [5]:
# 使用tf.keras.sequential构建模型
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28,28]))
model.add(keras.layers.Dense(300,activation='relu'))
model.add(keras.layers.Dense(100,activation='tanh'))
model.add(keras.layers.Dense(10,activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',optimizer='sgd',
             metrics=['accuracy'])

In [8]:
# Tensorboard , earlystopping 
logdir = './callbacks'
if not os.path.exists(logdir):
    os.mkdir(logdir)

output_model_file = os.path.join(logdir,"fashion_minist_model.h5")
callbacks = [
    keras.callbacks.TensorBoard(logdir),
    keras.callbacks.ModelCheckpoint(output_model_file,save_best_only=True),
    keras.callbacks.EarlyStopping(patience=5, min_delta = 1e-3)
]
history = model.fit(x_train,y_train,epochs=50,validation_data=(x_vaild,y_vaild),callbacks = callbacks)

Train on 55000 samples, validate on 5000 samples
Epoch 1/50
55000/55000 [==============================] - 8s 149us/sample - loss: 0.5485 - accuracy: 0.8108 - val_loss: 0.4168 - val_accuracy: 0.8550
Epoch 2/50
55000/55000 [==============================] - 5s 97us/sample - loss: 0.3957 - accuracy: 0.8583 - val_loss: 0.3719 - val_accuracy: 0.8682
Epoch 3/50
55000/55000 [==============================] - 6s 101us/sample - loss: 0.3566 - accuracy: 0.8729 - val_loss: 0.3543 - val_accuracy: 0.8738
Epoch 4/50
55000/55000 [==============================] - 5s 95us/sample - loss: 0.3319 - accuracy: 0.8813 - val_loss: 0.3412 - val_accuracy: 0.8748
Epoch 5/50
55000/55000 [==============================] - 5s 98us/sample - loss: 0.3133 - accuracy: 0.8868 - val_loss: 0.3304 - val_accuracy: 0.8826
Epoch 6/50
55000/55000 [==============================] - 5s 94us/sample - loss: 0.2977 - accuracy: 0.8927 - val_loss: 0.3200 - val_accuracy: 0.8826
Epoch 7/50
55000/55000 [==============================]

In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize = (8, 5))
    plt.grid(True)
    plt.gca().set_ylim(0, 1)
    plt.show()
    
plot_learning_curves(history)